In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()


True

## load variables
- data folder
- cleaned_df
- json template
- 

In [19]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
cleaned_df = pd.read_csv(cleaned_csv)
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'cleaned_trees_grasses_halfmoons.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)




### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [4]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = '''select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;'''
query2 = '''select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);'''
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

#df

## extract repeat csv records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [7]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
#halfmoon_csv = 'Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #step3: save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 14 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 81 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 26 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 99 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records
💯Created: cleaned_cleaned_trees_grasses_halfmoons.csv 🎯with 212 records


## enumerator
get user details from db for json population

In [14]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    i['id'] = idx
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_number': '2

## farmingEntity
- read column values from cleaned df and append to json keys

In [24]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
#ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 2,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 3,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'concern',
                     'id': 4,
                     'last_name': 'concern',
                     'middle_name': 'concern',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 5,
                     'last_name': 'acf',
                   

## plot

In [ ]:
import uuid
plot_details = template[0]['plot']
keys = 

#assign every value to none
for x in plot_details:
    plot_details[x] = None

plot_details_json_list = []
count = 0
#assign values from cleaned df    
for i in cleaned_df.to_dict(orient='records'):
    count += 1
    #ic(i)
    plot_details['name'] = uuid.uuid4().hex
    plot_details['area'] = i['area_ha']
    plot_details['photo_url'] = "https://radrs.icraf.org/uploaded/media/" + i['plot_details-photo']
    #ic(plot_details)
    ic(count)
    plot_details_json_list.append(plot_details)
ic(plot_details_json_list)



ic| count: 1
ic| count: 2
ic| count: 3
ic| count: 4
ic| count: 5
ic| count: 6
ic| count: 7
ic| count: 8
ic| count: 9
ic| count: 10
ic| count: 11
ic| count: 12
ic| count: 13
ic| count: 14
ic| count: 15
ic| count: 16
ic| count: 17
ic| count: 18
ic| count: 19
ic| count: 20
ic| count: 21
ic| count: 22
ic| count: 23
ic| count: 24
ic| count: 25
ic| count: 26
ic| count: 27
ic| count: 28
ic| count: 29
ic| count: 30
ic| count: 31
ic| count: 32
ic| count: 33
ic| count: 34
ic| count: 35
ic| count: 36
ic| count: 37
ic| count: 38
ic| count: 39
ic| count: 40
ic| count: 41
ic| count: 42
ic| count: 43
ic| count: 44
ic| count: 45
ic| count: 46
ic| count: 47
ic| count: 48
ic| count: 49
ic| count: 50
ic| count: 51
ic| count: 52
ic| count: 53
ic| count: 54
ic| count: 55
ic| count: 56
ic| count: 57
ic| count: 58
ic| count: 59
ic| count: 60
ic| count: 61
ic| count: 62
ic| count: 63
ic| count: 64
ic| count: 65
ic| count: 66
ic| count: 67
ic| count: 68
ic| count: 69
ic| count: 70
ic| count: 71
ic| count: 72
i

[{'id': None,
  'name': 'a206a02d80654c819571a39292f7e23a',
  'farmingEntityId': None,
  'has_crops': None,
  'fenced': None,
  'fence_type': None,
  'other_fence_type': None,
  'plot_ownership_type': None,
  'calculated_size': None,
  'other_ownership_type': None,
  'livestock_allowed': None,
  'conservation_area': None,
  'photo_url': 'https://radrs.icraf.org/uploaded/media/1751876154095.jpg',
  'subCountyId': None,
  'crops': None,
  'points': None,
  'subCounty': None,
  'area': 4.908980892},
 {'id': None,
  'name': 'a206a02d80654c819571a39292f7e23a',
  'farmingEntityId': None,
  'has_crops': None,
  'fenced': None,
  'fence_type': None,
  'other_fence_type': None,
  'plot_ownership_type': None,
  'calculated_size': None,
  'other_ownership_type': None,
  'livestock_allowed': None,
  'conservation_area': None,
  'photo_url': 'https://radrs.icraf.org/uploaded/media/1751876154095.jpg',
  'subCountyId': None,
  'crops': None,
  'points': None,
  'subCounty': None,
  'area': 4.90898089

#### halfmoon csv - repeat csv
goal: get plot level information from the repeat csv
- get unique rows based on `PARENT_KEY`
- based on this, populate plot details for columns
    - `has_crops`

In [22]:
unique_hm_df = halfmoon_df.drop_duplicates(subset=['PARENT_KEY'], keep='first')
unique_hm_df['crops_present']

0       no
1       no
2       no
7       no
10      no
11      no
12      no
13      no
14      no
17     yes
22     yes
28     yes
30     yes
38     yes
39     yes
40     yes
41     yes
42     yes
43     yes
44     yes
45     yes
46     yes
47     yes
48     yes
49     yes
50     yes
51     yes
52     yes
53     yes
54     yes
56     yes
62     yes
65     yes
68     NaN
72     NaN
75     NaN
78     NaN
118    NaN
120    NaN
124    NaN
126    NaN
128    NaN
129    NaN
134    NaN
143    NaN
163    NaN
167    NaN
170    NaN
173    NaN
176    NaN
181    NaN
189    NaN
198    NaN
206    NaN
209    NaN
Name: crops_present, dtype: object

populate the plot level info to the plot_details dictionary

- this will be based on id matching from unique(55) to main(119)

In [41]:
count = 0
for i in unique_hm_df.to_dict(orient='records'):
    count += 1
    if i['PARENT_KEY'] in keys:
        plot_details['has_crops'] = i['crops_present']
        
        ic(plot_details)
    ic(count)

ic| plot_details: {'area': 4.908980892,
                   'calculated_size': None,
                   'conservation_area': None,
                   'crops': None,
                   'farmingEntityId': None,
                   'fence_type': None,
                   'fenced': None,
                   'has_crops': 'no',
                   'id': None,
                   'livestock_allowed': None,
                   'name': '9eeb2b059bbb4d04a079b018cb87d31f',
                   'other_fence_type': None,
                   'other_ownership_type': None,
                   'photo_url': 'https://radrs.icraf.org/uploaded/media/1751876154095.jpg',
                   'plot_ownership_type': None,
                   'points': None,
                   'subCounty': None,
                   'subCountyId': None}
ic| count: 1
ic| plot_details: {'area': 4.908980892,
                   'calculated_size': None,
                   'conservation_area': None,
                   'crops': None,
                